In [39]:


import pandas as pd
data = pd.read_csv('Assault_Dataset.csv', encoding='ISO-8859-1')





C:\Users\risha\AppData\Local\Temp\ipykernel_7304\440063689.py:2: DtypeWarning: Columns (0,1,5,6,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259

# Loading and initial cleaning

In [40]:


threshold = 0.5  # 50% threshold for missing values
columns_to_drop = data.isnull().sum()[data.isnull().sum() > threshold * len(data)].index
data_cleaned = data.drop(columns=columns_to_drop)




In [41]:
# Replacing Placeholder Values ('.') with NaN && Removing Columns with More Than 90% Missing Values

In [42]:


import numpy as np
data_cleaned = data_cleaned.replace('.', np.nan)


threshold_for_deletion = 0.9  # 90% threshold for missing values
columns_to_delete = data_cleaned.columns[data_cleaned.isna().mean() > threshold_for_deletion]
data_final_cleaned = data_cleaned.drop(columns=columns_to_delete)

# Data Transformation (Standardization and One-Hot Encoding)


In [43]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

numerical_features = data_final_cleaned.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = data_final_cleaned.select_dtypes(include=['object']).columns.tolist()

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_transformed = preprocessor.fit_transform(data_final_cleaned)

# Preprocessing pipelines

In [44]:
# Saving the Final Cleaned Dataset to a CSV File

final_cleaned_data_file_path = 'final_cleaned_rochester_assault_data.csv'
data_final_cleaned.to_csv(final_cleaned_data_file_path, index=False)




# Applying preprocessing

In [45]:
data_processed = pd.read_csv(final_cleaned_data_file_path)


C:\Users\risha\AppData\Local\Temp\ipykernel_7304\736509032.py:1: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,19,20,22,23,24,25,26,33,34,37,45,47,48,49,50,51,52,53,68,81,84,86,87,88,89,90,93,94,95,96,97,98,99,100,104,107,108,109,110,111,112,113,114,115,116,119,120,121,122,160,161,165,166,167,168,172,173,174,175,176,177,178,179,180,191,195,196,197,198,199,200,201,202,203,204,207,208,209,210,211,229,252,253,254,255,256,260,261,262,263,266,267,268) have mixed types. Specify dtype option on import or set low_memory=False.
  data_processed = pd.read_csv(final_cleaned_data_file_path)


In [46]:
# Separate Numeric and Categorical Data

In [47]:
numeric_data = data_processed.select_dtypes(include=['int64', 'float64'])
categorical_data = data_processed.select_dtypes(exclude=['int64', 'float64'])


In [48]:
#Impute Missing Values in Numeric Data

In [49]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
numeric_data_imputed = pd.DataFrame(imputer.fit_transform(numeric_data), columns=numeric_data.columns)


In [50]:
# Scale Numeric Data

In [51]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numeric_data_scaled = pd.DataFrame(scaler.fit_transform(numeric_data_imputed), columns=numeric_data.columns)


In [52]:
# Apply PCA to Numeric Data

In [53]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
numeric_data_pca = pca.fit_transform(numeric_data_scaled)
pca_columns = ['PCA_Component_' + str(i) for i in range(numeric_data_pca.shape[1])]
numeric_data_pca_df = pd.DataFrame(numeric_data_pca, columns=pca_columns)


In [54]:
# Combine PCA-Transformed Numeric Data with Categorical Data

In [55]:
data_combined = pd.concat([numeric_data_pca_df.reset_index(drop=True), categorical_data.reset_index(drop=True)], axis=1)


In [56]:
# Save the Processed Dataset

In [57]:
processed_data_file_path = 'processed_rochester_assault_data.csv'
data_combined.to_csv(processed_data_file_path, index=False)
